# Notebook used to apply standard compression methods to Kodak dataset image for evaluation
## Importing all the necessary packages and initializing input and output directories

In [ ]:
import os
import ntpath
from PIL import Image
from pathlib import Path
import glob
import timeit
import subprocess
from numpy import mean,var
from datetime import datetime

vvenc_path = Path('/home/filippo/Downloads/vvenc-1.9.1/install/bin/vvencapp').resolve()
print ("VVEncAPP Path: ", str(vvenc_path))
vvdec_path = Path('/home/filippo/Downloads/vvdec-2.1.2/install/bin/vvdecapp').resolve()
print ("VVDevAPP Path: ", str(vvdec_path))

current_path = Path('.').resolve()
print("Current path: ",str(current_path))
source_path = str(current_path)+'/IMAGES/PNG_IMAGES'
source_path.replace(os.sep,ntpath.sep)
print("Source: ",source_path)

# JPEG conversion

## Setting output directory for JPEG file conversion

In [ ]:
destination_path = str(current_path)+'/IMAGES/JPEG/'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

## Converting images into JPEG format

In [ ]:
quality_index = [2,6,19,23,30] # The image quality, on a scale from 0 (worst) to 95 (best)
# 2=0.170bpp 3=0.174bpp 6=0.203bpp 19=0.327bpp 30=0.426bpp [2,6,19,23,30]| 
compression_times = {}
for qi in quality_index: 
    destination_path = str(current_path)+'/IMAGES/JPEG/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    if os.path.isdir(destination_path) & os.path.exists(destination_path):
        try:
            files = os.listdir(destination_path)
            for file in files:
                file_path = os.path.join(destination_path, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path)
    
    destination_path_JPEG = str(current_path)+'/IMAGES/JPEG/'+str(qi)+'/JPEG'
    destination_path_JPEG.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_JPEG)
    if os.path.isdir(destination_path_JPEG) & os.path.exists(destination_path_JPEG):
        try:
            files = os.listdir(destination_path_JPEG)
            for file in files:
                file_path = os.path.join(destination_path_JPEG, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path_JPEG)

    times = []    
    for file in glob.glob(str(source_path)+'/*.png'):
        image = Image.open(file)
        image = image.convert('RGB')
        starttime = timeit.default_timer()
        image.save(str(destination_path_JPEG)+'/'+Path(file).stem+'.jpeg','jpeg',subsampling=1, quality=qi)
        endtime = timeit.default_timer()
        execution_time = endtime-starttime
        times.append(execution_time)
        print('Encoded',Path(file).stem,' | Execution time: ',str(execution_time),' seconds')
        image.close()

    compression_times[qi]=times

## Converting JPEG compressed images back to PNG for later comparison

In [ ]:
for qi in quality_index:
    destination_path = str(current_path)+'/IMAGES/JPEG/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    destination_path_JPEG = str(current_path)+'/IMAGES/JPEG/'+str(qi)+'/JPEG'
    destination_path_JPEG.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_JPEG)
    
    for file in glob.glob(str(destination_path_JPEG)+'/*.jpeg'):
        image = Image.open(file)
        image = image.convert('RGB')
        image.save(str(destination_path)+'/'+Path(file).stem+'.png','png')
        print(Path(file).stem)
        image.close()

## Compute bits per pixel of converted images

In [ ]:
for qi in quality_index:
    destination_path = str(current_path)+'/IMAGES/JPEG/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    destination_path_JPEG = str(current_path)+'/IMAGES/JPEG/'+str(qi)+'/JPEG'
    destination_path_JPEG.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_JPEG)
    bpp_list=[]
    count = 0
    log = open(str(destination_path)+'/log.txt', 'w')
    log.writelines([str(datetime.now()),'\n'])
    for file in glob.glob(str(destination_path_JPEG)+'/*.jpeg'):
        image = Image.open(file)
        file_size = os.path.getsize(file) * 8
        pixels = image.width * image.height
        bits_per_pixel = file_size/pixels
        bpp_list.append(bits_per_pixel)
        log_string = 'Image:' + str(Path(file).stem) + '.jpeg\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(image.width) + '\tHeight:' + str(image.height) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[qi][count])
        log.writelines([log_string,'\n'])
        print(log_string)
        image.close()
        count+=1
        
    log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' ± ' + str(var(bpp_list)) + ' in ' + str(mean(compression_times[qi]))  + ' ± ' + str(var(compression_times[qi])) + ' s')])    
    print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' ± ' + str(var(bpp_list)) + ' in ' + str(mean(compression_times[qi])) + ' ± ' + str(var(compression_times[qi])) + ' s')
    log.close()

# JPEG2000 Conversion
## Setting output directory for JPEG2000 file conversion

In [ ]:
destination_path = str(current_path)+'/IMAGES/JPEG2000'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

## Converting Images to JPEG2000

In [ ]:
quality_layers = [175,140,115,72,56] # 175=0.137bpp  140 = 0.171bpp 115=0.208bpp 72=0.333bpp 56=0.428bpp
compression_times = {}
for qi in quality_layers:
    destination_path = str(current_path)+'/IMAGES/JPEG2000/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    if os.path.isdir(destination_path) & os.path.exists(destination_path):
        try:
            files = os.listdir(destination_path)
            for file in files:
                file_path = os.path.join(destination_path, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path)
    
    destination_path_JPEG2000 = str(current_path)+'/IMAGES/JPEG2000/'+str(qi)+'/JPEG2000'
    destination_path_JPEG2000.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_JPEG2000)
    if os.path.isdir(destination_path_JPEG2000) & os.path.exists(destination_path_JPEG2000):
        try:
            files = os.listdir(destination_path_JPEG2000)
            for file in files:
                file_path = os.path.join(destination_path_JPEG2000, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path_JPEG2000)
    
    times = []
    for file in glob.glob(str(source_path)+'/*.png'):
        image = Image.open(file)
        image = image.convert('RGB')
        starttime = timeit.default_timer()
        image.save(str(destination_path_JPEG2000)+'/'+Path(file).stem+'.jp2','jpeg2000',irreversible=True, quality_mode='rates',quality_layers=[qi])
        endtime = timeit.default_timer()
        execution_time = endtime-starttime
        times.append(execution_time)
        print('Encoded',Path(file).stem,' | Execution time: ',execution_time,' seconds')
        image.close()

    compression_times[qi]=times

## Converting JPEG200 compressed images back to PNG for later comparison

In [ ]:
for qi in quality_layers:
    destination_path = str(current_path)+'/IMAGES/JPEG2000/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    destination_path_JPEG2000 = str(current_path)+'/IMAGES/JPEG2000/'+str(qi)+'/JPEG2000'
    destination_path_JPEG2000.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_JPEG2000)

    for file in glob.glob(str(destination_path_JPEG2000)+'/*.jp2'):
        image = Image.open(file)
        image = image.convert('RGB')
        image.save(str(destination_path)+'/'+Path(file).stem+'.png','png')
        print(Path(file).stem)
        image.close()

## Compute bits per pixel of converted images

In [ ]:
for qi in quality_layers:
    destination_path = str(current_path)+'/IMAGES/JPEG2000/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    destination_path_JPEG2000 = str(current_path)+'/IMAGES/JPEG2000/'+str(qi)+'/JPEG2000'
    destination_path_JPEG2000.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_JPEG2000)
    bpp_list=[]
    count = 0
    log = open(str(destination_path)+'/log.txt', 'w')
    log.writelines([str(datetime.now()),'\n'])
    
    for file in glob.glob(str(destination_path_JPEG2000)+'/*.jp2'):
        image = Image.open(file)
        file_size = os.path.getsize(file) * 8
        pixels = image.width * image.height
        bits_per_pixel = file_size/pixels
        bpp_list.append(bits_per_pixel)
        log_string = 'Image:' + str(Path(file).stem) + '.jp2\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(image.width) + '\tHeight:' + str(image.height) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[qi][count])
        log.writelines([log_string,'\n'])
        print(log_string)
        image.close()
        count +=1
    
    log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' ± ' + str(var(bpp_list)) + ' in ' + str(mean(compression_times[qi])) + ' ± ' + str(var(compression_times[qi])) + ' s')])
    print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' ± ' + str(var(bpp_list)) + ' in ' + str(mean(compression_times[qi])) + ' ± ' + str(var(compression_times[qi])) + ' s')
    log.close()

# BPG Conversion
## Setting output directory for BPG conversion

In [ ]:
destination_path = str(current_path)+'/IMAGES/BPG'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

## Converting Images to BPG

In [ ]:
compression_times = {}
quantizer_parameters = [49,44,41,38,36] #set quantizer parameter (smaller gives better quality,range: 0-51, default = 29)
# 49=0.065bpp 44=0.134bpp 41=0.205bpp 38=0.313bpp 36=0.418bpp
for qi in quantizer_parameters:
    destination_path = str(current_path)+'/IMAGES/BPG/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    if os.path.isdir(destination_path) & os.path.exists(destination_path):
        try:
            files = os.listdir(destination_path)
            for file in files:
                file_path = os.path.join(destination_path, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path)
    
    destination_path_BPG = str(current_path)+'/IMAGES/BPG/'+str(qi)+'/BPG'
    destination_path_BPG.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_BPG)
    if os.path.isdir(destination_path_BPG) & os.path.exists(destination_path_BPG):
        try:
            files = os.listdir(destination_path_BPG)
            for file in files:
                file_path = os.path.join(destination_path_BPG, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path_BPG)
    times=[]    
    for file in os.listdir(source_path):
        source_file = str(source_path + '/' + file)
        source_file.replace(os.sep,ntpath.sep)
        destination_file = str(destination_path_BPG + '/' + str(Path(file).stem) + '.bpg')
        destination_file.replace(os.sep,ntpath.sep)
        starttime = timeit.default_timer()
        subprocess.run(['bpgenc','-c','rgb','-f','420','-m','1','-q',str(qi),'-o',str(destination_file),str(source_file)])
        endtime = timeit.default_timer()
        execution_time = endtime - starttime
        times.append(execution_time)
        print('Encoded', Path(file).stem, ' | Execution time: ', str(execution_time), ' seconds')
        
    compression_times[qi]=times

## Converting BPG compressed images back to PNG for later comparison

In [ ]:
for qi in quantizer_parameters:
    destination_path = str(current_path)+'/IMAGES/BPG/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    destination_path_BPG = str(current_path)+'/IMAGES/BPG/'+str(qi)+'/BPG'
    destination_path_BPG.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_BPG)
    
    for file in os.listdir(destination_path_BPG):
        source_file = str(destination_path_BPG + '/' + file)
        source_file.replace(os.sep,ntpath.sep)
        destination_file = str(destination_path + '/' + str(Path(file).stem) + '.png')
        destination_file.replace(os.sep,ntpath.sep)
        subprocess.run(['bpgdec','-o',str(destination_file),str(source_file)])
        print(Path(file).stem)

## Compute bits per pixel of converted images

In [ ]:
for qi in quantizer_parameters:
    destination_path = str(current_path)+'/IMAGES/BPG/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    destination_path_BPG = str(current_path)+'/IMAGES/BPG/'+str(qi)+'/BPG'
    destination_path_BPG.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_BPG)
    bpp_list=[]
    count = 0
    log = open(str(destination_path)+'/log.txt', 'w')
    log.writelines([str(datetime.now()),'\n'])
    for file in os.listdir(destination_path_BPG):
        filepath = str(destination_path_BPG) + '/' + str(file)
        filepath.replace(os.sep,ntpath.sep)
        file_size = os.path.getsize(filepath) * 8
        output = subprocess.Popen(['bpgdec','-i',str(filepath)],stdout=subprocess.PIPE).communicate()
        dimensions = (((str(output).split(' '))[0].split('='))[1].split('x'))
        pixels = int(dimensions[0]) * int(dimensions [1])
        bits_per_pixel = file_size/pixels
        bpp_list.append(bits_per_pixel)
        log_string = 'Image:' + str(Path(file).stem) + '.bpg\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(dimensions[0]) + '\tHeight:' + str(dimensions[1]) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[qi][count])
        log.writelines([log_string,'\n'])
        print(log_string)
        count +=1
    
    log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' ± ' + str(var(bpp_list)) + ' in ' + str(mean(compression_times[qi])) + ' ± ' + str(var(compression_times[qi])) + ' s')])
    print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' ± ' + str(var(bpp_list)) + ' in ' + str(mean(compression_times[qi])) + ' ± ' + str(var(compression_times[qi])) + ' s')
    log.close()

# VVC Conversion
## Setting input and output directory for VVC conversion

In [ ]:
source_path = str(current_path)+'/IMAGES/YUV_IMAGES'
source_path.replace(os.sep,ntpath.sep)
print("Source: ",source_path)

destination_path = str(current_path)+'/IMAGES/VVC'
destination_path.replace(os.sep,ntpath.sep)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

## Converting images to VVC

In [ ]:
compression_times = {}
quantizer_parameters = [42,36,33,31,30] #Quantization parameter (0...63) 0=better quality
# 42=0.662bpp 36=0.143bpp 33=0.227bpp 31=0.333bpp 30=0.405bpp
for qi in quantizer_parameters:
    destination_path = str(current_path)+'/IMAGES/VVC/'+str(qi)
    destination_path.replace(os.sep,ntpath.sep)
    if os.path.isdir(destination_path) & os.path.exists(destination_path):
        try:
            files = os.listdir(destination_path)
            for file in files:
                file_path = os.path.join(destination_path, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path)
    
    destination_path = str(current_path)+'/IMAGES/VVC/'+str(qi)+'/YUV'
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    if os.path.isdir(destination_path) & os.path.exists(destination_path):
        try:
            files = os.listdir(destination_path)
            for file in files:
                file_path = os.path.join(destination_path, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path)
    
    destination_path_VVC = str(current_path)+'/IMAGES/VVC/'+str(qi)+'/VVC'
    destination_path_VVC.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_VVC)
    if os.path.isdir(destination_path_VVC) & os.path.exists(destination_path_VVC):
        try:
            files = os.listdir(destination_path_VVC)
            for file in files:
                file_path = os.path.join(destination_path_VVC, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path_VVC)
    
    log_path = str(current_path)+'/IMAGES/VVC/'+str(qi)
    source_path.replace(os.sep,ntpath.sep)
    source_path.replace(os.sep,ntpath.sep)
    print("Log: ",log_path)
    log = open(str(log_path)+'/psnr_yuv.txt', 'w')
    times = []
    psnr_list = []
    for file in os.listdir(source_path):
        if file.lower().endswith('.yuv'):
            dimensions = open(source_path+'/'+str(Path(file).stem)+'.txt').readline()
            source_file = str(source_path + '/' + file)
            source_file.replace(os.sep,ntpath.sep)
            destination_file = str(destination_path_VVC + '/' + str(Path(file).stem) + '.vvc')
            destination_file.replace(os.sep,ntpath.sep)
            starttime = timeit.default_timer()
            output = subprocess.Popen([str(vvenc_path),'--profile','main_10_still_picture','-f','1','-c','yuv420_10','-s',str(dimensions),'--qp',str(qi),'--input',str(source_file),'--output',str(destination_file)],stdout=subprocess.PIPE).communicate()
            endtime = timeit.default_timer()
            execution_time = endtime - starttime - 0.2
            psnr = str(output).split("\\n")[20].split("  ")[11].strip()
            psnr_list.append(float(psnr))
            log.writelines([psnr,'\n'])
            times.append(execution_time)
            print('Encoded', Path(file).stem, ' | Execution time: ', str(execution_time), ' seconds','\n')
            
    compression_times[qi]=times

## Converting VVC Images back to YUV for later comparison
To convert images back to PNG refer to ImagePreparation

In [ ]:
for qi in quantizer_parameters:
    destination_path = str(current_path)+'/IMAGES/VVC/'+str(qi)+'/YUV'
    destination_path.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path)
    destination_path_VVC = str(current_path)+'/IMAGES/VVC/'+str(qi)+'/VVC'
    destination_path_VVC.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_VVC)
    
    for file in os.listdir(destination_path_VVC):
        source_file = str(destination_path_VVC + '/' + file)
        source_file.replace(os.sep,ntpath.sep)
        destination_file = str(destination_path+ '/' + str(Path(file).stem) + '.yuv')
        destination_file.replace(os.sep,ntpath.sep)
        subprocess.run([str(vvdec_path),'--bitstream',str(source_file),'--output',str(destination_file)])
        print('Decoded', Path(file).stem,'\n\n')

## Compute bits per pixel of cf converted images

In [ ]:
for qi in quantizer_parameters:
    destination_path_VVC = str(current_path)+'/IMAGES/VVC/'+str(qi)+'/VVC'
    destination_path_VVC.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_VVC)
    log_path = str(current_path)+'/IMAGES/VVC/'+str(qi)
    source_path.replace(os.sep,ntpath.sep)
    print("Log: ",log_path)
    bpp_list=[]
    count = 0
    log = open(str(log_path)+'/log.txt', 'w')
    log.writelines([str(datetime.now()),'\n'])
    for file in os.listdir(destination_path_VVC):
        filepath = str(destination_path_VVC) + '/' + str(file)
        filepath.replace(os.sep,ntpath.sep)
        file_size = os.path.getsize(filepath) * 8
        dimensions = open(source_path+'/'+str(Path(file).stem)+'.txt').readline().split('x')
        pixels = int(dimensions[0]) * int(dimensions [1])
        bits_per_pixel = file_size/pixels
        bpp_list.append(bits_per_pixel)
        log_string = 'Image:' + str(Path(file).stem) + '.vvc\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(dimensions[0]) + '\tHeight:' + str(dimensions[1]) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[qi][count])
        log.writelines([log_string,'\n'])
        print(log_string)
        count +=1
    
    log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' ± ' + str(var(bpp_list)) + ' in ' + str(mean(compression_times[qi])) + ' ± ' + str(var(compression_times[qi])) + ' s')])
    print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' ± ' + str(var(bpp_list)) + ' in ' + str(mean(compression_times[qi])) + ' ± ' + str(var(compression_times[qi])) + ' s')
    log.close()